In [1]:
import pandas as pd
import numpy as np
from catnip.fla_redshift import FLA_Redshift
from sqlalchemy import null
from datetime import datetime

from prefect.blocks.system import Secret
from typing import Dict
from concurrent.futures import ThreadPoolExecutor

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [2]:
def get_redshift_credentials() -> Dict:

    cred_dict = {
        "dbname": Secret.load("stellar-redshift-db-name").get(),
        "host": Secret.load("stellar-redshift-host").get(),
        "port": 5439,
        "user": Secret.load("stellar-redshift-user-name").get(),
        "password": Secret.load("stellar-redshift-password").get(),

        "aws_access_key_id": Secret.load("fla-s3-aws-access-key-id-east-1").get(),
        "aws_secret_access_key": Secret.load("fla-s3-aws-secret-access-key-east-1").get(),
        "bucket": Secret.load("fla-s3-bucket-name-east-1").get(),
        "subdirectory": "us-east-1",

        "verbose": False,
    }

    return cred_dict

with ThreadPoolExecutor(1) as pool:
    rs_creds = pool.submit(lambda: get_redshift_credentials()).result()

In [3]:
q = """
with attendance as
    (select
        event_datetime,
        count(*) as attendance
    from
        custom.cth_v_attendance_2324
    group by
        event_datetime
    UNION ALL
    select
        event_datetime,
        count(*) as attendance
    from
        custom.cth_v_attendance_2425
    group by
        event_datetime)
select
    cth_game_descriptions.season,
    cth_game_descriptions.event_date,
    tier,
    is_premier,
    original_six_plus_extra,
    day_of_week,
    start_time,
    attendance,
    gross_revenue,
    num_orders,
    quantity_sold
from
    custom.cheq_v_hockey_summary
left join
    custom.cth_game_descriptions on date(cheq_v_hockey_summary.event_date) = date(cth_game_descriptions.event_date)
left join
    attendance on date(attendance.event_datetime) = date(cheq_v_hockey_summary.event_date)
where
    tier in ('A','B','C','D','E')
"""

historical_f_and_b = FLA_Redshift(**rs_creds).query_warehouse(sql_string = q)

In [5]:
day_map = {
    'Fri': 1,
    'Sat': 1,
    'Sun': 1,
    'Mon': 0,
    'Tue': 0,
    'Wed': 0,
    'Thu': 0
}

historical_f_and_b['weekend'] = historical_f_and_b.apply(
    lambda row: day_map.get(row['day_of_week'], 0), axis=1)

start_time_map = {
    '12:30 PM': 1,
    '12:45 PM': 1,
    '1:00 PM': 1,
    '3:00 PM': 1,
    '3:30 PM': 1,
    '4:00 PM': 2,
    '5:00 PM': 2,
    '6:00 PM': 2
}

historical_f_and_b['start_time_num'] = historical_f_and_b.apply(
    lambda row: start_time_map.get(row['start_time'], 0),
    axis=1)

tier_mapping = {
    'A': 5,
    'B': 4,
    'C': 3,
    'D': 2,
    'E': 1
}

historical_f_and_b['tier_num'] = historical_f_and_b.apply(
    lambda row: tier_mapping.get(row['tier'], 0),
    axis=1)

In [6]:
q = """
with attendance as
    (select
        event_datetime,
        count(*) as attendance
    from
        custom.cth_v_attendance_2324
    group by
        event_datetime
    UNION ALL
    select
        event_datetime,
        count(*) as attendance
    from
        custom.cth_v_attendance_2425
    group by
        event_datetime)
select
    cth_game_descriptions.season,
    cth_game_descriptions.event_date,
    tier,
    is_premier,
    original_six_plus_extra,
    day_of_week,
    start_time,
    attendance,
    sum(gross_revenue) as gross_revenue,
    sum(qty) as quantity,
    count(distinct invoice_id) as num_orders
from
    custom.retailpro_v_invoice_items
left join
    custom.cth_game_descriptions on retailpro_v_invoice_items.event_date = cth_game_descriptions.event_date
left join
    attendance on retailpro_v_invoice_items.event_date = date(attendance.event_datetime)
where
    season in ('2023-24','2024-25')
    and tier in ('A','B','C','D','E')
group by
    cth_game_descriptions.season,
    cth_game_descriptions.event_date,
    tier,
    is_premier,
    original_six_plus_extra,
    day_of_week,
    start_time,
    attendance
"""

historical_merch = FLA_Redshift(**rs_creds).query_warehouse(sql_string = q)

In [7]:
day_map = {
    'Fri': 1,
    'Sat': 1,
    'Sun': 1,
    'Mon': 0,
    'Tue': 0,
    'Wed': 0,
    'Thu': 0
}

historical_merch['weekend'] = historical_merch.apply(
    lambda row: day_map.get(row['day_of_week'], 0), axis=1)

start_time_map = {
    '12:30 PM': 1,
    '12:45 PM': 1,
    '1:00 PM': 1,
    '3:00 PM': 1,
    '3:30 PM': 1,
    '4:00 PM': 2,
    '5:00 PM': 2,
    '6:00 PM': 2
}

historical_merch['start_time_num'] = historical_merch.apply(
    lambda row: start_time_map.get(row['start_time'], 0),
    axis=1)

tier_mapping = {
    'A': 5,
    'B': 4,
    'C': 3,
    'D': 2,
    'E': 1
}

historical_merch['tier_num'] = historical_merch.apply(
    lambda row: tier_mapping.get(row['tier'], 0),
    axis=1)

In [8]:
q = """
select
    season,
    event_date,
    tier,
    is_premier,
    original_six_plus_extra,
    day_of_week,
    start_time
from
    custom.cth_game_descriptions
where
    event_date >= current_date
"""

future_game_info = FLA_Redshift(**rs_creds).query_warehouse(sql_string = q)

predicted_attendance = pd.read_csv('C:\\Users\\riffere\\Desktop\\output.csv')

future_game_info = future_game_info.merge(predicted_attendance, how = 'left', on = 'event_date')

In [9]:
day_map = {
    'Fri': 1,
    'Sat': 1,
    'Sun': 1,
    'Mon': 0,
    'Tue': 0,
    'Wed': 0,
    'Thu': 0
}

future_game_info['weekend'] = future_game_info.apply(
    lambda row: day_map.get(row['day_of_week'], 0), axis=1)

start_time_map = {
    '12:30 PM': 1,
    '12:45 PM': 1,
    '1:00 PM': 1,
    '3:00 PM': 1,
    '3:30 PM': 1,
    '4:00 PM': 2,
    '5:00 PM': 2,
    '6:00 PM': 2
}

future_game_info['start_time_num'] = future_game_info.apply(
    lambda row: start_time_map.get(row['start_time'], 0),
    axis=1)

tier_mapping = {
    'A': 5,
    'B': 4,
    'C': 3,
    'D': 2,
    'E': 1
}

future_game_info['tier_num'] = future_game_info.apply(
    lambda row: tier_mapping.get(row['tier'], 0),
    axis=1)

In [10]:
def run_f_and_b_model(df, df_future):

    x_train = df[['attendance','weekend','start_time_num','tier_num']]
    y_train = df[['gross_revenue']]

    x_test = df_future[['attendance','weekend','start_time_num','tier_num']]

    #scalar = PolynomialFeatures(degree=2, include_bias=False)
    #poly_features = scalar.fit_transform(x_train)
    scalar = StandardScaler()
    poly_features = scalar.fit_transform(x_train)

    polynomial = LinearRegression().fit(poly_features, np.array(y_train).ravel())
    #predicted_train = polynomial.predict(poly_features)

    poly_features2 = scalar.fit_transform(x_test)

    return polynomial.predict(poly_features2).astype(int)

In [11]:
future_game_info['predicted_f_and_b_gross_rev'] = run_f_and_b_model(historical_f_and_b, future_game_info)

In [12]:
def run_merch_model(df, df_future):

    x_train = df[['attendance','weekend','start_time_num','tier_num']]
    y_train = df[['gross_revenue']]

    x_test = df_future[['attendance','weekend','start_time_num','tier_num']]

    #scalar = PolynomialFeatures(degree=2, include_bias=False)
    #poly_features = scalar.fit_transform(x_train)
    scalar = StandardScaler()
    poly_features = scalar.fit_transform(x_train)

    polynomial = LinearRegression().fit(poly_features, np.array(y_train).ravel())
    #predicted_train = polynomial.predict(poly_features)

    poly_features2 = scalar.fit_transform(x_test)

    return polynomial.predict(poly_features2).astype(int)

In [13]:
future_game_info['predicted_merch_gross_rev'] = run_merch_model(historical_merch, future_game_info)